In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/train.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/valid.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/test.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
# Import OS module
import os

# Imports
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks, layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img,img_to_array
from keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from keras.applications.vgg19 import VGG19 , preprocess_input , decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Notebook magic
%matplotlib inline

In [ ]:
# Configure variables for Transfer learning
image_size = 224
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)
grid_shape = (1, image_size, image_size, 3)

batch_size = 32

In [ ]:
train_dir = "/content/train"
test_dir = "/content/test"
valid_dir = "/content/valid"

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
valid_data= valid_datagen.flow_from_directory(
    valid_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Disable shuffling for test data
)
test_data= test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Disable shuffling for test data
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Found 1812 images belonging to 38 classes.


**MobileNet V2**

In [ ]:
# Load the base model
mbnet_v2 = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape
)

# Stop from being trainable
mbnet_v2.trainable = False

In [ ]:
model = Sequential()
model.add(mbnet_v2)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(len(train_data.class_indices), activation="softmax"))

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss for early stopping
    patience=5,  # Number of epochs with no improvement after which training will stop
    restore_best_weights=True  # Restore the model with the best validation loss
)

In [ ]:
num_epochs = 30
history=model.fit(
    train_data,
    epochs=num_epochs,
    steps_per_epoch = 200,
    validation_data=valid_data,
    callbacks=[early_stopping]
)

Epoch 1/30
200/200 [==============================] - 119s 566ms/step - loss: 2.7730 - accuracy: 0.2523 - val_loss: 2.2803 - val_accuracy: 0.3807
Epoch 2/30
200/200 [==============================] - 112s 558ms/step - loss: 2.0388 - accuracy: 0.4441 - val_loss: 2.0469 - val_accuracy: 0.4355
Epoch 3/30
200/200 [==============================] - 114s 569ms/step - loss: 1.7641 - accuracy: 0.5042 - val_loss: 1.8832 - val_accuracy: 0.4678
Epoch 4/30
200/200 [==============================] - 114s 573ms/step - loss: 1.6503 - accuracy: 0.5280 - val_loss: 1.7195 - val_accuracy: 0.5023
Epoch 5/30
200/200 [==============================] - 111s 556ms/step - loss: 1.5525 - accuracy: 0.5455 - val_loss: 1.7304 - val_accuracy: 0.5027
Epoch 6/30
200/200 [==============================] - 120s 602ms/step - loss: 1.4910 - accuracy: 0.5695 - val_loss: 1.5808 - val_accuracy: 0.5321
Epoch 7/30
200/200 [==============================] - 112s 559ms/step - loss: 1.4309 - accuracy: 0.5831 - val_loss: 1.5655 -

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print("Test accuracy:", test_accuracy)

In [ ]:
hist = history.history
# Plot accuracy and loss
plt.plot(hist["accuracy"], label="accuracy")
plt.plot(hist["val_accuracy"], label="val_accuracy")

# Add the labels and legend
plt.ylabel("Accuracy / Loss")
plt.xlabel("Epochs #")
plt.legend()

plt.show()

In [ ]:
if "val_accuracy" in hist and "val_loss" in hist:
    plt.plot(hist["loss"], label="loss")
    plt.plot(hist["val_loss"], label="val_loss")

# Add the labels and legend
plt.ylabel("Accuracy / Loss")
plt.xlabel("Epochs #")
plt.legend()

plt.show()

In [ ]:
model.save("/content/drive/MyDrive/MobileNet.h5")

In [ ]:
import json

from IPython.display import FileLink

# Link the files
FileLink("./content/drive/MyDrive/MobileNet.h5")

In [ ]:
# Dump the categories
with open("categories.json", "w") as file:
  json.dump(train_data.class_indices, file)

# Link categories JSON
FileLink("categories.json")

In [ ]:
class_mapping = {
    0: 'Apple Scab',
    1: 'Apple Black Rot',
    2: 'Apple Rust',
    3: 'Apple Healthy',
    4: 'Blueberry Healthy',
    5: 'Cherry (including sour) Powdery mildew',
    6: 'Cherry (including sour) healthy',
    7: 'Corn (maize) Cercospora leaf spot Gray leaf spot',
    8: 'Corn (maize) Common rust',
    9: 'Corn (maize) Northern Leaf Blight',
    10: 'Corn (maize) Healthy',
    11: 'Grape Black Rot',
    12: 'Grape Esca (Black Measles)',
    13: 'Grape Leaf blight (Isariopsis Leaf Spot)',
    14: 'Grape Healthy',
    15: 'Orange Haunglongbing (Citrus greening)',
    16: 'Peach Bacterial Spot',
    17: 'Peach Healthy',
    18: 'Pepper Bell Bacterial Spot',
    19: 'Pepper Bell Healthy',
    20: 'Potato Early Blight',
    21: 'Potato Late Blight',
    22: 'Potato Healthy',
    23: 'Raspberry Healthy',
    24: 'Soyabean Healthy',
    25: 'Squash Powdery Mildew',
    26: 'Strawberry Leaf Scorch',
    27: 'Strawberry Healthy',
    28: 'Tomato Bacterial Spot',
    29: 'Tomato Early Blight',
    30: 'Tomato Late Blight',
    31: 'Tomato Leaf Mold',
    32: 'Tomato Septoria leaf spot',
    33: 'Tomato Spider mites Two-spotted spider mite',
    34: 'Tomato Target Spot',
    35: 'Tomato Tomato Yellow Leaf Curl Virus',
    36: 'Tomato Tomato mosaic virus',
    37: 'Tomato Healthy',
    38: 'None',
    }

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

image_path = '/content/test/test_data/AppleCedarRust4.JPG'
img = image.load_img(image_path, target_size=(224, 224))  # Adjust target_size as needed
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [ ]:
predictions = model.predict(img_array)

In [ ]:
predicted_class_index = np.argmax(predictions)
print(predicted_class_index)
predicted_disease_class = class_mapping.get(predicted_class_index, 'Unknown')
print("Predicted Disease Class:", predicted_disease_class)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
y_pred = model.predict(valid_data, steps=len(valid_data), verbose=1)

In [ ]:
class_indices = valid_data.class_indices

In [ ]:
y_true = [class_indices[filename.split('/')[0]] for filename in valid_data.filenames]

In [ ]:
class_accuracies = []

for class_label in range(38):
    class_indices = [i for i, label in enumerate(y_true) if label == class_label]

    # Check if there are samples for this class
    if not class_indices:
        accuracy = 0.0  # No samples for this class, accuracy is 0
    else:
        class_y_true = [y_true[i] for i in class_indices]
        class_y_pred = [y_pred[i] for i in class_indices]
        accuracy = accuracy_score(class_y_true, np.argmax(class_y_pred, axis=1))

    class_accuracies.append(accuracy)

# Print class-specific accuracies
for i, accuracy in enumerate(class_accuracies):
    print(f'Class {i}: Accuracy = {accuracy:.2f}')
